In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import json
import logging

In [2]:
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",
    "Content-Type": "text/plain;charset=UTF-8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "cs-CZ,cs;q=0.9,en;q=0.8,sk;q=0.7"
}

In [4]:
data = []

In [5]:
def parseArticle(soup, href):
    title = soup.find("h1").text
    author = soup.find("div", {"rel":"rnews:creator schema:creator"}).find("a").text
    published = soup.find("meta", {"itemprop":"datePublished"})["content"]
    modified = soup.find("meta", {"itemprop":"dateModified"})["content"]
    score = soup.find("section", {"class":"field field-name-field-readability-score field-type-list-float field-label-inline clearfix view-mode-_custom_display"}).find("div", {"class":"field-item even"}).text.strip()

    item = {
        'title': title,
        'author': author,
        'link': href,
        'published': published,
        'modified': modified,
        'score': score
    }
    
    #print(item)
    
    data.append(item)

In [6]:
def crawler(seed):
    frontier=[seed]
    crawled=[]
    while frontier:
        page=frontier.pop()
        try:
            if page not in crawled:
                print(page)
                sleep(10)
                source = requests.get(page).text
                soup=BeautifulSoup(source, "html5lib")
                if '/article/' in page:
                    parseArticle(soup, page)
                links=soup.findAll('a',href=True)
                for link in links:
                    new_link = link['href']
                    if(new_link.startswith(seed)):
                        frontier.append(new_link)
                crawled.append(page)
 
        except Exception as e:
            print(e)
    return crawled

In [7]:
crawler("https://www.sciencenewsforstudents.org/")

https://www.sciencenewsforstudents.org/
https://www.sciencenewsforstudents.org/feeds/all.rss
https://www.sciencenewsforstudents.org/how-to-use
https://www.sciencenewsforstudents.org/collections/zika
https://www.sciencenewsforstudents.org/women-girls-science-STEM
https://www.sciencenewsforstudents.org/collections/solar-eclipse-2017
https://www.sciencenewsforstudents.org/collections/kid-scientists
https://www.sciencenewsforstudents.org/collections/invention-innovation
https://www.sciencenewsforstudents.org/collections/diy-science-snot-science
https://www.sciencenewsforstudents.org/collections/diy-science-5-second-rule
https://www.sciencenewsforstudents.org/collections/cool-jobs
https://www.sciencenewsforstudents.org/collections/concussions
https://www.sciencenewsforstudents.org/blog/analyze/
https://www.sciencenewsforstudents.org/about-us
https://www.sciencenewsforstudents.org/author/sarah-zielinski
https://www.sciencenewsforstudents.org/article/explainer-weather-and-weather-prediction
h

KeyboardInterrupt: 

In [8]:
data

[{'author': 'Sarah Zielinski',
  'link': 'https://www.sciencenewsforstudents.org/article/explainer-weather-and-weather-prediction',
  'modified': '7:00AM, October 19, 2017',
  'published': '7:00AM, October 19, 2017',
  'score': '7.5',
  'title': 'Explainer: Weather and weather prediction'},
 {'author': 'Carolyn Gramling',
  'link': 'https://www.sciencenewsforstudents.org/article/defining-dinosaur-harder-ever',
  'modified': '6:45AM, March 1, 2018',
  'published': '6:45AM, March 1, 2018',
  'score': '8.2',
  'title': 'Defining a dinosaur is now far harder'},
 {'author': 'Kathiann Kowalski',
  'link': 'https://www.sciencenewsforstudents.org/article/can-dna-editing-save-endangered-species',
  'modified': '6:45AM, March 6, 2018',
  'published': '6:45AM, March 6, 2018',
  'score': '6.6',
  'title': 'Can DNA editing save endangered species?'},
 {'author': 'Janet Raloff',
  'link': 'https://www.sciencenewsforstudents.org/article/vaping-may-harm-lungs',
  'modified': '1:50PM, August 15, 2017',

In [9]:
with open('out.json', 'w') as outfile:
    json.dump(data, outfile)